# Remote controling the Bot via MQTT in a notebook

First imports to drive the bot. Don't forget to run the notebook from where a virtual environment with the mqtt client is installed.

## MQTT INIT

In [1]:
import paho.mqtt.client as mqtt
import time
import json
import numpy
from numpy import pi
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)

0

In [4]:
client.ws_set_options()

## BRICK PI
#### CHANGE LOGGING LEVELS

In [4]:
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)
level = 30
payload_dict = {
    'ddr_odometry': level,
#     'obj_detector_run_detection' : 30,
#     'async_run_capture_loop': 10
#     'obj_detector_async_record_video': level,
#     'obj_detector_async_upload_file_to_azure': level
}
payload = json.dumps(payload_dict)
msg = client.publish(topic='bot/logger/multiple', payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid)) 

MSG rc/mess_id : 0/1


## JETSON

#### Setting Logging Levels

In [22]:
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)
level = 10
payload_dict = {
    'obj_detector_mqtt_loop': level,
    'obj_detector_run_detection' : 30,
    'async_run_capture_loop': 10
}
payload = json.dumps(payload_dict)
msg = client.publish(topic='bot/logger/multiple', payload=payload, qos=1)

#### Configuration Object Settings

In [ ]:
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)
payload = {'show_video': True,
           'max_unseen_time_for_object': 2,
           'time_between_scoring_service_calls': 1,
           'detection_threshold': 0.5,
           'max_tracked_objects': 1}
payload = json.dumps(payload)
msg = client.publish(topic="bot/jetson/configure", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

#### Streaming Video

In [3]:
client = mqtt.Client(client_id="commander", clean_session=True)
client.connect("192.168.32.50", 1883, keepalive=60)
payload = {'target_ip': "192.168.32.55",
           'target_port': 5555}
payload = json.dumps(payload)
msg = client.publish(topic="bot/jetson/video_streaming/start", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/1


In [7]:
msg = client.publish(topic="bot/jetson/video_streaming/stop", payload=json.dumps({}), qos=1)

In [6]:
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)
payload = {'duration': 5}
payload = json.dumps(payload)
msg = client.publish(topic="bot/jetson/start_video", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/1


In [5]:
client = mqtt.Client(client_id="commander", clean_session=True)   # , transport='websockets')
client.connect("192.168.32.50", 1883, keepalive=60)
payload = {'include_visualisation_data':True}
payload = json.dumps(payload)
msg = client.publish(topic="bot/jetson/snap_picture", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/1


### Set End Effector in Pick Position

In [181]:
moveforwardpayload = {}
moveforwardpayload["P"] = [0.2, 0.00, -0.1]      # wrt camera...
moveforwardpayload["velocity_factor"] = 0.075
moveforwardpayload["pid"] = True
moveforwardpayload["grip_angle"] = 90 * 2 * pi / 360
moveforwardpayload["control_method"] = 'velocity'
moveforwardpayload["dryrun"] = False
#moveforwardpayload["override"] = True
payload = json.dumps(moveforwardpayload)
msg = client.publish(topic="bot/gripper/position", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/2


## TESTED ASYNC OK - Set ARM in theta positions (theta1 and theta2)

In [13]:
jsonpayload = {}
jsonpayload["theta1"] = 0 * (2 * pi / 360)
jsonpayload["theta2"] = 0 * (2 * pi / 360)
jsonpayload["dryrun"] = False
jsonpayload["velocity_factor"] = 0.75
payload = json.dumps(jsonpayload)
msg = client.publish(topic="bot/arm/move/theta", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/11


In [12]:
jsonpayload = {}
jsonpayload["theta1"] = 40 * (2 * pi / 360)
jsonpayload["theta2"] = 94 * (2 * pi / 360)
jsonpayload["dryrun"] = False
jsonpayload["velocity_factor"] = 1
payload = json.dumps(jsonpayload)
msg = client.publish(topic="bot/arm/move/theta", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/10


## TESTED ASYNC OK - Close Gripper

In [7]:
payload = json.dumps({})
msg = client.publish(topic="bot/gripper/close", payload=payload, qos=1)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))

MSG rc/mess_id : 0/2


## TESTED ASYNC OK - Open Gripper 

In [8]:
payload = json.dumps({})
msg = client.publish(topic="bot/gripper/open", payload=payload, qos=1)
print("MSG published : {}/{}".format(msg.rc, msg.mid))

MSG published : 0/3


# TESTED ASYNC OK - Go Home

In [103]:
phipayload = {}
phipayload["control_method"] = 'velocity'
phipayload["velocity_factor"] = 0.1
phipayload["dryrun"] = False
# phipayload["override"] = True
payload = json.dumps(phipayload)
msg = client.publish(topic="bot/base/move/home", payload=payload, qos=0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - Turn Phi

In [147]:
phipayload = {}
phipayload["phi"] = -90 * (2 * pi / 360)
phipayload["pid"] = True
phipayload["velocity_factor"] = 0.15
phipayload["dryrun"] = False
# phipayload["inter_wheel_distance"] = 0.246
payload = json.dumps(phipayload)
msg = client.publish(topic="bot/base/turn/phi", payload=payload, qos=0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - Set ARM in resting position (to drive the robot)

In [10]:
jsonpayload = {}
jsonpayload["dryrun"] = False
jsonpayload["velocity_factor"] = 0.1
payload = json.dumps(jsonpayload)
msg = client.publish(topic="bot/arm/move/rest", payload=payload, qos=0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - Set ARM in theta zero position

In [47]:
jsonpayload = {}
jsonpayload["dryrun"] = False
jsonpayload["velocity_factor"] = 0.1
payload = json.dumps(jsonpayload)
msg = client.publish(topic="bot/arm/move/zero", payload=payload, qos=0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - MOVE XYPHI

In [9]:
jsonpayload = {}
jsonpayload["x"] = 0.1
jsonpayload["y"] = 0 # -0.25
jsonpayload["phi"] = 0 * (2 * pi / 360)
jsonpayload["pid"] = True
jsonpayload["velocity_factor"] = 0.075
jsonpayload["dryrun"] = False
payload = json.dumps(jsonpayload)
msg = client.publish(topic="bot/base/move/xyphi", payload=payload, qos=0)
print("MSG rc/mess_id : {}/{}".format(msg.rc, msg.mid))
# print("MSG published : {}".format(msg.is_published()))

MSG rc/mess_id : 0/4


## TESTED ASYNC OK - MOVE X

In [160]:
moveforwardpayload = {}
moveforwardpayload["distance"] = -0.10
moveforwardpayload["pid"] = True  # True   # False
moveforwardpayload["velocity_factor"] = 0.075
moveforwardpayload["dryrun"] = False
moveforwardpayload["wheel_radius"] = 0.0341
payload = json.dumps(moveforwardpayload)
msg = client.publish(topic="bot/base/move/x", payload=payload, qos=0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - RESET COORDINATES

In [173]:
resetpayload = {}
resetpayload["now"] = 0
payload = json.dumps(resetpayload)
msg = client.publish(topic='bot/base/reset/position', payload=payload, qos = 0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True


## TESTED ASYNC OK - KILL SWITCH!

In [23]:
killswitchpayload = {}
payload = json.dumps(killswitchpayload)
msg = client.publish(topic='bot/killSwitch', payload=payload, qos = 0)
print("MSG published : {}".format(msg.is_published()))

MSG published : True
